In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet(
    "/home/pl487/anchoral/outputs/dry_run/agnews/random_2023-08-30T12-39-29/tb_logs.parquet"
)

In [25]:
import torch
from torch.utils.data import DataLoader, RandomSampler

seed = 0
g = torch.Generator()

dataset = list(range(10))

g.manual_seed(seed)

sampler = RandomSampler(dataset, generator=g)

dl = DataLoader(dataset, batch_size=1, sampler=sampler)
a = list(dl)

g.manual_seed(seed)
b = DataLoader(dataset, batch_size=1, sampler=sampler)

In [31]:
dl = DataLoader(dataset, batch_size=1, shuffle=True)
dl2 = DataLoader(dataset, batch_size=1, shuffle=True)

a = list(dl)
b = list(dl2)
c = list(dl)

list(zip(a, b, c))

[(tensor([3]), tensor([2]), tensor([8])),
 (tensor([4]), tensor([0]), tensor([3])),
 (tensor([8]), tensor([3]), tensor([5])),
 (tensor([9]), tensor([8]), tensor([9])),
 (tensor([0]), tensor([9]), tensor([6])),
 (tensor([2]), tensor([7]), tensor([0])),
 (tensor([5]), tensor([1]), tensor([4])),
 (tensor([1]), tensor([5]), tensor([2])),
 (tensor([6]), tensor([4]), tensor([1])),
 (tensor([7]), tensor([6]), tensor([7]))]